In [27]:
import pandas as pd
import numpy as np
td = pd.read_csv('titanic-data.csv')
td.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 先看看数据集的一些基本统计情况

In [45]:
def blingbling(df, filter, col_name, kv):
    df2 = df
    if filter:
        df2 = df[filter]
    one_col = df2[col_name]
    total_count = one_col.size
    msgs = []
    for key in kv:
        percent = 1.0*one_col[one_col==key].size / total_count
        msgs.append('%s: %f'%(kv[key], percent))
    print " ; ".join(msgs)

blingbling(td, None, 'Sex', {'male':'男性占比', 'female':'女性占比'})
blingbling(td, None, 'Survived', {0:'不在', 1:'幸存'})

男性占比: 0.647587 ; 女性占比: 0.352413
不在: 0.616162 ; 幸存: 0.383838


In [17]:
kv={"a":"aaa","b":"bbb"}
for k in kv:
    print k

a
b


－－－－－－－－－－－－－－－below are temp

In [43]:
df2 = pd.DataFrame({ 'A' : 1.,
    'B' : pd.Timestamp('20130102'),
    'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
    'D' : np.array([3] * 4,dtype='int32'),
    'E' : pd.Categorical(["test","train","test","train"]),
    'F' : 'foo' })
df2[True]

KeyError: True